In [1]:
import os
out_dir = 'C:/temp/bcu/out'
out_file_name = 'bcu-theses-out.xml'
out_file = os.path.normpath('{}/{}'.format(out_dir, out_file_name))

log_dir = 'C:/temp/bcu/log'
log_file = 'bcu-transform-theses.log'

In [2]:
import logging
log_file = os.path.join(log_dir, log_file)
file_mode = 'w'
logging_level = logging.DEBUG
logging.basicConfig(filename=log_file, filemode=file_mode, level=logging_level, format='%(asctime)s %(levelname)s : %(message)s')
PRINT_STACKTRACE = True

In [3]:
# Credentials to download file from eprints
eprints_creds_file = os.path.normpath('C:/Users/MohapatraD/.secrets/BCU/eprints-credentials.txt')

with open(eprints_creds_file, 'r') as ecf:
    EPRINTS_CREDS = ecf.read().rstrip()

In [4]:
#%run ./mapping_functions.ipynb

In [5]:
%run ./Mapping_student_thesis.ipynb

In [6]:
#%run ./Mapping_dataset.ipynb

In [7]:
DATABASE = 'BCU'
DB_USER = 'postgres'
DB_PASSWORD = 'admin'

import psycopg2
conn = psycopg2.connect(host="localhost", port="5432", database=DATABASE, user=DB_USER, password=DB_PASSWORD)
conn.autocommit = True
sql = "select id, type, xml_in from items where xml_out is null and type in ('thesis')"
sql2 = 'update items set xml_out = %s where id = %s'
with conn:
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    logging.info('Loaded all publications from the database')
    for row in rows:
        id = row[0]
        content_type = row[1]
        xml_in = row[2]
        logging.info('Transforming item {} of type {}'.format(id, content_type))
        try:
            xml_out = map(id, content_type, xml_in)
            cur.execute(sql2, (xml_out, id))
        except Exception as e:
            if PRINT_STACKTRACE:
                logging.exception('Error transforming publication {} - {}'.format(id, repr(e)))
            else:
                logging.error('Error transforming publication {} - {}'.format(id, repr(e)))
conn.close() 
logging.info('All items processed')

In [8]:
conn = psycopg2.connect(host="localhost", port="5432", database=DATABASE, user=DB_USER, password=DB_PASSWORD)
# Collect all results
with open(out_file, 'w', encoding='utf-8') as out:
    out.write('{}\n'.format(XML_HEADER))
    cur = conn.cursor()
    cur.execute("select xml_out from items where xml_out is not null")
    rows = cur.fetchall()
    for row in rows:
        out.write('{}\n'.format(row[0]))
    out.write(XML_FOOTER)
    logging.info('XML import file saved at "{}"'.format(out_file))
    cur.close()